In [29]:
import pandas as pd
import sqlite3 as lite

# Database connection
engine = lite.connect('store1.db')

def generate_store1_2024_sales_report():
    # Query to get sales data for StoreKey 1 in Q1 2024 only
    query = """
    SELECT 
        p.SKU,
        p.ProductName as product_name,
        p.Category as category,
        p.Subcategory as subcategory,
        SUM(sf.'#SoldToday') as sales
    FROM 
        SalesFact_DailyLevel sf
    JOIN 
        ProductDimension p ON sf.ProductKey = p.ProductKey
    JOIN 
        DateDimension d ON sf.DateKey = d.DateKey
    WHERE 
        sf.StoreKey = 1
        AND d.Year = 2024
        AND d.Quarter = 1
    GROUP BY 
        p.SKU, p.ProductName, p.Category, p.Subcategory
    """

    # Load data into DataFrame
    df = pd.read_sql(query, engine)

    # Calculate ranks
    df['2024 Q1 Rank'] = df['sales'].rank(
        ascending=False, method='min').astype(int)

    # Rename columns
    df = df.rename(columns={'sales': '2024 Q1 Sales'})

    # Sort by rank
    df = df.sort_values('2024 Q1 Rank')

    # Select top 25 based on 2024 Q1 sales
    top_products = df.head(25)

    # Select and order final columns
    final_report = top_products[[
        'SKU', 'product_name', 'category', 'subcategory',
        '2024 Q1 Rank', '2024 Q1 Sales'
    ]]

    # Convert sales to integers
    final_report['2024 Q1 Sales'] = final_report['2024 Q1 Sales'].astype(int)

    return final_report


# Generate and save the report
report2024Q1 = generate_store1_2024_sales_report()
report2024Q1

,SKU,product_name,category,subcategory,2024 Q1 Rank,2024 Q1 Sales
277,42358001,2.00% Milk,Dairy,Milk,1,62638752
274,42355001,1.00% Milk,Dairy,Milk,2,60419100
279,42360001,Whole Milk Milk,Dairy,Milk,3,60311820
278,42359001,Whole Milk Milk,Dairy,Milk,4,59871120
276,42357001,2.00% Milk,Dairy,Milk,5,59168580
275,42356001,1.00% Milk,Dairy,Milk,6,58272168
230,42311001,Jam Grape,Jams and Jellies,Jam,7,5215752
232,42313001,Jelly Grape,Jams and Jellies,Jam,8,4790640
231,42312001,Jam Strawberry,Jams and Jellies,Jam,9,4756452
233,42314001,Squeeze Jelly Grape,Jams and Jellies,Jelly,10,4536876


In [30]:
report2024Q1_SKU = report2024Q1[['SKU']].copy()

In [31]:
def generate_store1_2025_sales_report():
    # Query to get sales data for StoreKey 1 in Q1 2025 only
    query = f"""
    SELECT 
        p.SKU,
        p.ProductName as product_name,
        p.Category as category,
        p.Subcategory as subcategory,
        SUM(sf.'#SoldToday') as sales
    FROM 
        SalesFact_DailyLevel sf
    JOIN 
        ProductDimension p ON sf.ProductKey = p.ProductKey
    JOIN 
        DateDimension d ON sf.DateKey = d.DateKey
    WHERE 
        sf.StoreKey = 1
        AND d.Year = 2025
        AND d.Quarter = 1
        AND p.SKU IN ({','.join(report2024Q1_SKU['SKU'].astype(str))})
    GROUP BY 
        p.SKU, p.ProductName, p.Category, p.Subcategory
    """

    # Load data into DataFrame
    df = pd.read_sql(query, engine)

    # Calculate ranks
    df['2025 Q1 Rank'] = df['sales'].rank(
        ascending=False, method='min').astype(int)

    # Rename columns
    df = df.rename(columns={'sales': '2025 Q1 Sales'})

    # Sort by rank
    df = df.sort_values('2025 Q1 Rank')

    # Select top 25 based on 2025 Q1 sales
    top_products = df.head(25)

    # Select and order final columns
    final_report = top_products[[
        'SKU', 'product_name', 'category', 'subcategory',
        '2025 Q1 Rank', '2025 Q1 Sales'
    ]]

    # Convert sales to integers
    final_report['2025 Q1 Sales'] = final_report['2025 Q1 Sales'].astype(int)

    return final_report


# Generate and save the report
report2025Q1 = generate_store1_2025_sales_report()
report2025Q1

,SKU,product_name,category,subcategory,2025 Q1 Rank,2025 Q1 Sales
10,42359001,Whole Milk Milk,Dairy,Milk,1,537192024
8,42357001,2.00% Milk,Dairy,Milk,2,533311176
11,42360001,Whole Milk Milk,Dairy,Milk,3,527949372
9,42358001,2.00% Milk,Dairy,Milk,4,527624976
6,42355001,1.00% Milk,Dairy,Milk,5,527490360
7,42356001,1.00% Milk,Dairy,Milk,6,523447488
2,42311001,Jam Grape,Jams and Jellies,Jam,7,42131400
5,42314001,Squeeze Jelly Grape,Jams and Jellies,Jelly,8,41812020
4,42313001,Jelly Grape,Jams and Jellies,Jam,9,40699680
3,42312001,Jam Strawberry,Jams and Jellies,Jam,10,39737808


In [32]:
# Merge reports on SKU
final_report = pd.merge(
    report2024Q1,
    report2025Q1[['SKU', '2025 Q1 Rank', '2025 Q1 Sales']],
    on='SKU',
    how='left'
)

In [34]:
# Fill missing values for products not sold in 2025
final_report['2025 Q1 Sales'] = final_report['2025 Q1 Sales'].fillna(
    0).astype(int)
final_report['2025 Q1 Rank'] = final_report['2025 Q1 Rank'].fillna(
    len(final_report) + 1).astype(int)

# Select and order final columns
final_report = final_report[[
    'SKU', 'product_name', 'category', 'subcategory',
    '2024 Q1 Rank', '2024 Q1 Sales', '2025 Q1 Rank', '2025 Q1 Sales'
]]

# Save reports
final_report.to_csv('store1_sales_comparison.csv', index=False)

print("Final Combined Report:")
final_report

Final Combined Report:


,SKU,product_name,category,subcategory,2024 Q1 Rank,2024 Q1 Sales,2025 Q1 Rank,2025 Q1 Sales
0,42358001,2.00% Milk,Dairy,Milk,1,62638752,4,527624976
1,42355001,1.00% Milk,Dairy,Milk,2,60419100,5,527490360
2,42360001,Whole Milk Milk,Dairy,Milk,3,60311820,3,527949372
3,42359001,Whole Milk Milk,Dairy,Milk,4,59871120,1,537192024
4,42357001,2.00% Milk,Dairy,Milk,5,59168580,2,533311176
5,42356001,1.00% Milk,Dairy,Milk,6,58272168,6,523447488
6,42311001,Jam Grape,Jams and Jellies,Jam,7,5215752,7,42131400
7,42313001,Jelly Grape,Jams and Jellies,Jam,8,4790640,9,40699680
8,42312001,Jam Strawberry,Jams and Jellies,Jam,9,4756452,10,39737808
9,42314001,Squeeze Jelly Grape,Jams and Jellies,Jelly,10,4536876,8,41812020
